In [11]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [14]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [16]:
openai = OpenAI()


In [17]:
message = "Hello GPT! This is my first messeage to you. Was Up! "
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! I'm glad to hear from you! I'm here to help with whatever you need. What’s on your mind today?


In [31]:
# A class to represent a webpage

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


In [ ]:
king = Website("https://portfself.vercel.app/")
print(king.title)
print(king.text)


In [32]:
# Define ou system promt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."





In [40]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(king))


You are looking at a website titled DLCT PIXE SYSTEM
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Alain De La Cuadra
Fullstack Developer Specializing in Frontend
Designar och implementerar användarvänliga och responsiva gränssnitt som fascinerar och engagerar. Min entusiasm för frontend kompletteras av en bred förståelse för hela stacken, vilket gör att jag kan bygga robusta och skalbara webblösningar. Ständig utveckling och samarbete är kärnan i mitt arbete.
CLONA OCH KÖR PÅ!
Pusselbitar och Programvara
AI Generate Course
Kräver egen API-nyckel – funkar med Gemini ,OpenAi.
Skapa egna AI-drivna utbildningar med Youtube.
GitHub Repo
Visa på LinkedIn
AI Wireframe to Code
Omvandla skisser till kod. Kräver Gemini API-nyckel.
Testa hur AI tolkar design och genererar färdig kod.
GitHub Repo
Visa på LinkedIn
LMS System
Online kurs byggd i Next.js – responsiv och skalbar.

In [41]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

    

In [44]:
messages_for(king)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled DLCT PIXE SYSTEM\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nAlain De La Cuadra\nFullstack Developer Specializing in Frontend\nDesignar och implementerar användarvänliga och responsiva gränssnitt som fascinerar och engagerar. Min entusiasm för frontend kompletteras av en bred förståelse för hela stacken, vilket gör att jag kan bygga robusta och skalbara webblösningar. Ständig utveckling och samarbete är kärnan i mitt arbete.\nCLONA OCH KÖR PÅ!\nPusselbitar och Programvara\nAI Generate Course\nKräver egen API-nyckel – funkar med Gemini ,OpenAi.\nSkapa egna AI-drivna utbildningar med Youtube.\nGitHub R

In [48]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
         model = "gpt-4o-mini",
         messages= messages_for(website)
    )
    return response.choices[0].message.content

In [50]:
summarize("https://portfself.vercel.app/")

"# DLCT PIXE SYSTEM\n\nThe DLCT PIXE SYSTEM website showcases Alain De La Cuadra, a Fullstack Developer specializing in frontend development. The site highlights his ability to design and implement user-friendly and responsive interfaces while understanding the complete technology stack, enabling him to create robust web solutions. Continuous development and collaboration are central to his work philosophy.\n\n## Key Features\n\n- **AI Generate Course**: A tool that allows the creation of AI-driven courses using YouTube, requiring an API key for Gemini or OpenAI.\n- **AI Wireframe to Code**: A feature that converts sketches into code based on AI interpretation, also requiring a Gemini API key.\n- **LMS System**: An online course platform built with Next.js, which includes functionalities like course management, login, and progress tracking.\n\n## Tech Stack\nAlain showcases flexibility and a quick learning ability in various technologies, with interactive elements that reflect his pers

In [51]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [52]:
display_summary("https://portfself.vercel.app/")

# DLCT PIXE SYSTEM Summary

The DLCT PIXE SYSTEM website presents Alain De La Cuadra, a fullstack developer specializing in frontend development. Alain focuses on creating user-friendly and responsive interfaces, combining expertise across the entire tech stack to build robust web solutions.

## Key Offerings:
- **AI Generate Course:** Tools to create AI-driven courses using YouTube and requires an API key for Gemini or OpenAI.
- **AI Wireframe to Code:** Converts design sketches into code, also requiring a Gemini API key.
- **LMS System:** A scalable online course platform built with Next.js that includes features like user login and progress tracking.

## Skillset:
Alain showcases a flexible tech stack and a quick learning ability for various tools and technologies. His profile incorporates user interaction elements to engage visitors.

## Testimonials:
Several satisfied clients praise Alain's technical acumen, positive team spirit, and impactful solutions in LMS development and UX design.

## Services:
Alain offers tailored technical solutions aligned with business visions, emphasizing Learning Management Systems for effective training and onboarding. He utilizes advanced AI techniques to enhance data utility and customer interactions through customized chatbots and AI agents.

Overall, the website emphasizes Alain's commitment to continuous development, collaboration, and the incorporation of innovative technologies into web solutions.